# Iterative Reservoir INR: Image Generation as Temporal Process

**Key Idea**: Generate a spatial image through multiple iterative steps, treating it as a spatiotemporal process.

Instead of: `(x,y) → single forward pass → RGB`

We do: `(x,y) → step₁ → step₂ → ... → step_K → RGB`

This connects to:
- **Diffusion models** (iterative denoising)
- **Neural ODEs** (continuous dynamics)
- **Progressive refinement** (coarse-to-fine)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

try:
    import torch
    import torch.nn as nn
    USE_TORCH = True
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"PyTorch available, using {device}")
except ImportError:
    USE_TORCH = False
    print("PyTorch not available")

In [ ]:
# Load image
img = Image.open('fig/cat.png').convert('RGB')
target_size = 128  # Smaller for faster iteration
img = img.resize((target_size, target_size), Image.LANCZOS)
img_array = np.array(img) / 255.0

h, w, c = img_array.shape
coords = np.linspace(0, 1, h, endpoint=False)
x_grid = np.stack(np.meshgrid(coords, coords), -1)
X = x_grid.reshape(-1, 2)
Y = img_array.reshape(-1, 3)

print(f"Image: {h}x{w}, Samples: {len(X)}")
plt.imshow(img_array)
plt.title('Target')
plt.axis('off')
plt.show()

## Approach 1: Reservoir State Trajectory

Use ALL intermediate states, not just final:

```
h₀ = 0
h₁ = tanh(W_in·[x,y] + W_hh·h₀)
h₂ = tanh(W_in·[x,y] + W_hh·h₁)
...
features = [h₁, h₂, ..., h_K]  # Use entire trajectory
```

In [ ]:
def reservoir_trajectory(x, hidden_size, num_steps, spectral_radius=0.95):
    """
    Return the ENTIRE trajectory of reservoir states.
    Each step's state can be used to generate an image.
    """
    np.random.seed(42)
    n, d = x.shape
    
    # Initialize weights
    W_in = np.random.randn(d, hidden_size) * 0.5
    W_hh = np.random.randn(hidden_size, hidden_size)
    eig = np.abs(np.linalg.eigvals(W_hh)).max()
    W_hh = W_hh * (spectral_radius / eig)
    b = np.random.randn(hidden_size) * 0.1
    
    # Collect trajectory for all samples
    trajectories = []  # List of (n, hidden_size) arrays, one per step
    
    H = np.zeros((n, hidden_size))  # Current state
    
    for step in range(num_steps):
        H = np.tanh(x @ W_in + H @ W_hh + b)
        trajectories.append(H.copy())
    
    return trajectories, W_in, W_hh, b

# Test
trajectories, W_in, W_hh, b = reservoir_trajectory(X, hidden_size=256, num_steps=10)
print(f"Generated {len(trajectories)} trajectory steps, each shape: {trajectories[0].shape}")

In [ ]:
def ridge_regression(H, y, lamb=1e-6):
    W = np.linalg.solve(H.T @ H + lamb * np.eye(H.shape[1]), H.T @ y)
    return W

# Train a separate readout for each step
print("Training readout for each trajectory step...")
step_images = []
step_psnrs = []

for i, H in enumerate(trajectories):
    W = ridge_regression(H, Y)
    pred = np.clip(H @ W, 0, 1)
    mse = np.mean((pred - Y) ** 2)
    psnr = -10 * np.log10(mse)
    step_images.append(pred.reshape(h, w, 3))
    step_psnrs.append(psnr)
    print(f"  Step {i+1:2d}: PSNR = {psnr:.2f} dB")

In [ ]:
# Visualize the progressive refinement
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for i, ax in enumerate(axes.flat):
    if i < len(step_images):
        ax.imshow(step_images[i])
        ax.set_title(f'Step {i+1}\n{step_psnrs[i]:.1f} dB')
    ax.axis('off')

plt.suptitle('Reservoir Trajectory: Image at Each Step', fontsize=14)
plt.tight_layout()
plt.savefig('iterative_trajectory.png', dpi=150)
plt.show()

# Plot PSNR progression
plt.figure(figsize=(8, 4))
plt.plot(range(1, len(step_psnrs)+1), step_psnrs, 'bo-', markersize=8)
plt.xlabel('Iteration Step')
plt.ylabel('PSNR (dB)')
plt.title('Image Quality vs Reservoir Iteration')
plt.grid(True, alpha=0.3)
plt.show()

## Approach 2: Cumulative Trajectory Features

Instead of separate images per step, concatenate trajectory for richer features:

```
features = concat([h₁, h₂, ..., h_K])  # K×hidden_size features
```

In [ ]:
# Cumulative trajectory features
print("\nCumulative Trajectory Features:")
print("-" * 40)

cumulative_results = []

for k in [1, 2, 3, 5, 10]:
    # Concatenate first k steps
    H_cumulative = np.hstack(trajectories[:k])
    W = ridge_regression(H_cumulative, Y)
    pred = np.clip(H_cumulative @ W, 0, 1)
    mse = np.mean((pred - Y) ** 2)
    psnr = -10 * np.log10(mse)
    cumulative_results.append((k, psnr, pred.reshape(h, w, 3)))
    print(f"  Steps 1-{k:2d} (dim={H_cumulative.shape[1]:4d}): PSNR = {psnr:.2f} dB")

## Approach 3: Iterative Output Refinement (Diffusion-like)

Each step takes the PREVIOUS OUTPUT and refines it:

```
output₀ = initial_guess (or zeros)
output₁ = reservoir(x, y, output₀) 
output₂ = reservoir(x, y, output₁)
...
```

This is similar to diffusion models / iterative refinement.

In [ ]:
def iterative_refinement_reservoir(x, y, hidden_size=256, num_iterations=10, 
                                    spectral_radius=0.9):
    """
    Iterative refinement: each step refines the previous output.
    
    Input to reservoir at each step: [x, y, current_estimate]
    Output: refined_estimate
    """
    np.random.seed(42)
    n, d = x.shape
    out_dim = y.shape[1]  # 3 for RGB
    
    # Reservoir weights (input includes coordinates + current estimate)
    input_dim = d + out_dim  # x,y + RGB estimate
    W_in = np.random.randn(input_dim, hidden_size) * 0.5
    W_hh = np.random.randn(hidden_size, hidden_size)
    eig = np.abs(np.linalg.eigvals(W_hh)).max()
    W_hh = W_hh * (spectral_radius / eig)
    b_h = np.random.randn(hidden_size) * 0.1
    
    # Start with zeros (or could start with noise like diffusion)
    current_estimate = np.zeros((n, out_dim))
    h = np.zeros((n, hidden_size))
    
    iteration_outputs = []
    iteration_psnrs = []
    
    for it in range(num_iterations):
        # Combine coordinates with current estimate
        combined_input = np.hstack([x, current_estimate])
        
        # Reservoir update
        h = np.tanh(combined_input @ W_in + h @ W_hh + b_h)
        
        # Train readout for this iteration (or use fixed readout)
        W_out = ridge_regression(h, y)
        current_estimate = np.clip(h @ W_out, 0, 1)
        
        # Evaluate
        mse = np.mean((current_estimate - y) ** 2)
        psnr = -10 * np.log10(mse)
        
        iteration_outputs.append(current_estimate.reshape(h.shape[0]//w, w, out_dim))
        iteration_psnrs.append(psnr)
    
    return iteration_outputs, iteration_psnrs

print("Iterative Refinement (Diffusion-like):")
print("-" * 40)
refine_outputs, refine_psnrs = iterative_refinement_reservoir(X, Y, hidden_size=256, num_iterations=10)

for i, psnr in enumerate(refine_psnrs):
    print(f"  Iteration {i+1:2d}: PSNR = {psnr:.2f} dB")

In [ ]:
# Visualize iterative refinement
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for i, ax in enumerate(axes.flat):
    if i < len(refine_outputs):
        ax.imshow(refine_outputs[i])
        ax.set_title(f'Iter {i+1}\n{refine_psnrs[i]:.1f} dB')
    ax.axis('off')

plt.suptitle('Iterative Refinement: Diffusion-like Image Generation', fontsize=14)
plt.tight_layout()
plt.savefig('iterative_refinement.png', dpi=150)
plt.show()

## Approach 4: Learned Iterative Refinement (PyTorch)

Train a network that explicitly learns to refine over multiple steps.

In [ ]:
if USE_TORCH:
    class IterativeReservoirINR(nn.Module):
        """
        Learned iterative refinement for image generation.
        
        Each iteration:
        1. Takes (x, y, current_rgb_estimate)
        2. Updates hidden state with reservoir-like dynamics
        3. Outputs refined RGB estimate
        """
        def __init__(self, hidden_size=256, num_iterations=5):
            super().__init__()
            self.hidden_size = hidden_size
            self.num_iterations = num_iterations
            
            # Input projection: (x, y, r, g, b) → hidden
            self.W_in = nn.Linear(5, hidden_size)
            
            # Recurrent connection (like reservoir W_hh, but learned)
            self.W_hh = nn.Linear(hidden_size, hidden_size, bias=False)
            
            # Output projection: hidden → RGB
            self.W_out = nn.Linear(hidden_size, 3)
            
            # Optional: per-iteration refinement weights
            self.refine = nn.Sequential(
                nn.Linear(hidden_size, hidden_size),
                nn.ReLU(),
                nn.Linear(hidden_size, 3),
                nn.Sigmoid()
            )
        
        def forward(self, coords, return_trajectory=False):
            """
            coords: (N, 2) - spatial coordinates
            returns: (N, 3) - RGB values, or list of (N, 3) if return_trajectory
            """
            batch_size = coords.shape[0]
            
            # Initialize
            h = torch.zeros(batch_size, self.hidden_size, device=coords.device)
            rgb_estimate = torch.zeros(batch_size, 3, device=coords.device)
            
            trajectory = []
            
            for it in range(self.num_iterations):
                # Combine coords with current estimate
                combined = torch.cat([coords, rgb_estimate], dim=-1)  # (N, 5)
                
                # Reservoir-like update
                h = torch.tanh(self.W_in(combined) + self.W_hh(h))
                
                # Output refinement
                rgb_estimate = self.refine(h)
                
                if return_trajectory:
                    trajectory.append(rgb_estimate)
            
            if return_trajectory:
                return trajectory
            return rgb_estimate
    
    print("IterativeReservoirINR model defined")
else:
    print("PyTorch not available")

In [ ]:
if USE_TORCH:
    # Training
    model = IterativeReservoirINR(hidden_size=256, num_iterations=5).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    X_t = torch.FloatTensor(X).to(device)
    Y_t = torch.FloatTensor(Y).to(device)
    
    losses = []
    
    print("Training Iterative Reservoir INR...")
    for epoch in tqdm(range(1000)):
        optimizer.zero_grad()
        
        # Get trajectory of outputs
        trajectory = model(X_t, return_trajectory=True)
        
        # Loss on ALL iterations (encourages progressive refinement)
        loss = 0
        for it, pred in enumerate(trajectory):
            # Weight later iterations more
            weight = (it + 1) / len(trajectory)
            loss += weight * torch.mean((pred - Y_t) ** 2)
        
        loss.backward()
        optimizer.step()
        
        if epoch % 100 == 0:
            losses.append(loss.item())
    
    print(f"Final loss: {loss.item():.6f}")

In [ ]:
if USE_TORCH:
    # Visualize learned iterative refinement
    model.eval()
    with torch.no_grad():
        trajectory = model(X_t, return_trajectory=True)
    
    fig, axes = plt.subplots(1, len(trajectory) + 1, figsize=(15, 3))
    
    axes[0].imshow(img_array)
    axes[0].set_title('Target')
    axes[0].axis('off')
    
    for i, pred in enumerate(trajectory):
        pred_img = pred.cpu().numpy().reshape(h, w, 3)
        mse = np.mean((pred_img - img_array) ** 2)
        psnr = -10 * np.log10(mse)
        
        axes[i+1].imshow(pred_img)
        axes[i+1].set_title(f'Iter {i+1}\n{psnr:.1f} dB')
        axes[i+1].axis('off')
    
    plt.suptitle('Learned Iterative Refinement', fontsize=14)
    plt.tight_layout()
    plt.savefig('learned_iterative_refinement.png', dpi=150)
    plt.show()

## Approach 5: Multi-Scale Iterative (Coarse-to-Fine)

Different iterations focus on different frequency scales:
- Early iterations: low frequency (coarse structure)
- Later iterations: high frequency (fine details)

In [ ]:
def multiscale_iterative_reservoir(x, y, hidden_size=256, num_scales=5):
    """
    Each scale uses different spectral radius (controls frequency).
    Low spectral radius → smooth/low-freq
    High spectral radius → detailed/high-freq
    """
    n, d = x.shape
    
    # Different spectral radii for different scales
    spectral_radii = np.linspace(0.5, 0.99, num_scales)
    
    scale_outputs = []
    cumulative_output = np.zeros_like(y)
    
    for scale_idx, sr in enumerate(spectral_radii):
        np.random.seed(42 + scale_idx)
        
        # Reservoir for this scale
        W_in = np.random.randn(d, hidden_size) * 0.5
        W_hh = np.random.randn(hidden_size, hidden_size)
        eig = np.abs(np.linalg.eigvals(W_hh)).max()
        W_hh = W_hh * (sr / eig)
        b = np.random.randn(hidden_size) * 0.1
        
        # Run reservoir
        h = np.zeros((n, hidden_size))
        for _ in range(5):  # A few iterations per scale
            h = np.tanh(x @ W_in + h @ W_hh + b)
        
        # Predict RESIDUAL (what's missing from current estimate)
        residual_target = y - cumulative_output
        W_out = ridge_regression(h, residual_target)
        residual_pred = h @ W_out
        
        # Add to cumulative
        cumulative_output = np.clip(cumulative_output + residual_pred, 0, 1)
        
        mse = np.mean((cumulative_output - y) ** 2)
        psnr = -10 * np.log10(mse)
        scale_outputs.append((cumulative_output.copy(), psnr, sr))
        print(f"  Scale {scale_idx+1} (ρ={sr:.2f}): PSNR = {psnr:.2f} dB")
    
    return scale_outputs

print("\nMulti-Scale Iterative (Coarse-to-Fine):")
print("-" * 40)
multiscale_outputs = multiscale_iterative_reservoir(X, Y, hidden_size=256, num_scales=5)

In [ ]:
# Visualize multi-scale progression
fig, axes = plt.subplots(1, len(multiscale_outputs) + 1, figsize=(15, 3))

axes[0].imshow(img_array)
axes[0].set_title('Target')
axes[0].axis('off')

for i, (output, psnr, sr) in enumerate(multiscale_outputs):
    axes[i+1].imshow(output.reshape(h, w, 3))
    axes[i+1].set_title(f'Scale {i+1} (ρ={sr:.2f})\n{psnr:.1f} dB')
    axes[i+1].axis('off')

plt.suptitle('Multi-Scale Reservoir: Coarse-to-Fine Refinement', fontsize=14)
plt.tight_layout()
plt.savefig('multiscale_refinement.png', dpi=150)
plt.show()

## Comparison: All Approaches

In [ ]:
# Compare with Fourier baseline
def fourier_features(x, num_features, sigma):
    np.random.seed(42)
    B = np.random.randn(num_features, x.shape[1]) * sigma
    x_proj = (2. * np.pi * x) @ B.T
    return np.concatenate([np.sin(x_proj), np.cos(x_proj)], axis=-1)

H_fourier = fourier_features(X, 256, sigma=10)
W_fourier = ridge_regression(H_fourier, Y)
pred_fourier = np.clip(H_fourier @ W_fourier, 0, 1)
mse_fourier = np.mean((pred_fourier - Y) ** 2)
psnr_fourier = -10 * np.log10(mse_fourier)

print("\n" + "=" * 60)
print("COMPARISON: ALL APPROACHES")
print("=" * 60)
print(f"\nFourier Features (baseline):        {psnr_fourier:.2f} dB")
print(f"Reservoir Trajectory (best step):   {max(step_psnrs):.2f} dB")
print(f"Cumulative Trajectory (all steps):  {cumulative_results[-1][1]:.2f} dB")
print(f"Iterative Refinement (final):       {refine_psnrs[-1]:.2f} dB")
print(f"Multi-Scale (final):                {multiscale_outputs[-1][1]:.2f} dB")

In [ ]:
# Final comparison visualization
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

axes[0, 0].imshow(img_array)
axes[0, 0].set_title('Target', fontsize=12)
axes[0, 0].axis('off')

axes[0, 1].imshow(pred_fourier.reshape(h, w, 3))
axes[0, 1].set_title(f'Fourier\n{psnr_fourier:.1f} dB', fontsize=12)
axes[0, 1].axis('off')

best_step_idx = np.argmax(step_psnrs)
axes[0, 2].imshow(step_images[best_step_idx])
axes[0, 2].set_title(f'Best Trajectory Step\n{step_psnrs[best_step_idx]:.1f} dB', fontsize=12)
axes[0, 2].axis('off')

axes[0, 3].imshow(cumulative_results[-1][2])
axes[0, 3].set_title(f'Cumulative Trajectory\n{cumulative_results[-1][1]:.1f} dB', fontsize=12)
axes[0, 3].axis('off')

axes[1, 0].imshow(refine_outputs[-1])
axes[1, 0].set_title(f'Iterative Refinement\n{refine_psnrs[-1]:.1f} dB', fontsize=12)
axes[1, 0].axis('off')

axes[1, 1].imshow(multiscale_outputs[-1][0].reshape(h, w, 3))
axes[1, 1].set_title(f'Multi-Scale\n{multiscale_outputs[-1][1]:.1f} dB', fontsize=12)
axes[1, 1].axis('off')

# Plot progression curves
ax = axes[1, 2]
ax.plot(range(1, len(step_psnrs)+1), step_psnrs, 'o-', label='Trajectory')
ax.plot(range(1, len(refine_psnrs)+1), refine_psnrs, 's-', label='Iterative')
ax.plot(range(1, len(multiscale_outputs)+1), [x[1] for x in multiscale_outputs], '^-', label='Multi-Scale')
ax.axhline(psnr_fourier, color='r', linestyle='--', label='Fourier')
ax.set_xlabel('Step/Scale')
ax.set_ylabel('PSNR (dB)')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)
ax.set_title('Progression Curves')

axes[1, 3].axis('off')

plt.tight_layout()
plt.savefig('iterative_reservoir_comparison.png', dpi=150)
plt.show()

## Key Insights

In [ ]:
print("""
══════════════════════════════════════════════════════════════════════
                    KEY INSIGHTS: ITERATIVE RESERVOIR INR
══════════════════════════════════════════════════════════════════════

1. TRAJECTORY AS FEATURES
   ━━━━━━━━━━━━━━━━━━━━━━
   Using ALL reservoir states (not just final) provides richer features.
   Each step captures different aspects of the input.
   Cumulative trajectory often beats single-step.

2. ITERATIVE REFINEMENT
   ━━━━━━━━━━━━━━━━━━━━
   Like diffusion models: start with coarse estimate, refine progressively.
   Each iteration sees (coordinates + current_estimate) → refined_estimate.
   Natural way to incorporate feedback / error correction.

3. MULTI-SCALE / COARSE-TO-FINE
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   Different spectral radii → different frequency responses.
   Low ρ: smooth, coarse structure
   High ρ: detailed, fine structure
   Predicting residuals at each scale is effective.

4. WHEN ITERATIVE HELPS
   ━━━━━━━━━━━━━━━━━━━━
   ✓ When data has hierarchical structure (coarse → fine)
   ✓ When refinement makes sense conceptually
   ✓ For complex outputs requiring multiple "passes"
   ✗ For simple mappings, single-pass may suffice

5. CONNECTION TO OTHER METHODS
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━
   - Diffusion Models: iterative denoising
   - Neural ODEs: continuous-depth networks  
   - Recurrent refinement: iterative inference
   - Cascade networks: coarse-to-fine prediction

══════════════════════════════════════════════════════════════════════
""")